# Agent | Tools

**1. 도구 (Tool)**

https://docs.langchain.com/oss/python/integrations/tools/index#tools-and-toolkits

> "LLM이 사용할 수 있는 구체적인 기술이나 장비"

LLM은 기본적으로 학습된 데이터 내에서만 답변할 수 있으며, 실시간 정보나 정확한 수학 계산에는 취약하다. **Tool**은 이러한 한계를 보완하기 위해 LLM에게 쥐여주는 외부 기능이다.

* **역활:** 외부 API 호출, 웹 검색, 코드 실행, 파일 시스템 접근 등 LLM이 직접 할 수 없는 작업을 대신 수행한다.
* **예시:**
* `Google Search`: 최신 정보를 검색한다.
* `Calculator`: 정확한 수학 계산을 수행한다.
* `Python REPL`: 파이썬 코드를 작성하고 실행한다.



**2. 에이전트 (Agent)**

> "도구를 언제, 어떻게 사용할지 결정하는 두뇌"

**Agent**는 LLM을 추론 엔진(Reasoning Engine)으로 사용하여 사용자의 요청을 해결하기 위한 계획을 세우고 실행하는 주체이다. 단순히 정해진 코드를 순서대로 실행하는 것이 아니라, 상황에 따라 유연하게 행동을 결정한다.

* **역활:** 사용자의 질문을 분석하고, 어떤 **Tool**이 필요한지 판단(Thought)하고, 해당 도구를 실행(Action)한 뒤, 그 결과(Observation)를 보고 다음 행동을 결정하거나 최종 답변을 내놓는다.
* **작동 방식 (ReAct 패턴 예시):**
1. **질문:** "현재 서울 날씨에 맞는 옷차림 추천해줘."
2. **생각(Thought):** "서울의 현재 날씨를 먼저 알아야 한다." -> `Search` 도구 선택
3. **행동(Action):** `Search("서울 현재 날씨")` 실행
4. **관찰(Observation):** "서울 기온 5도, 맑음"이라는 결과 획득
5. **생각(Thought):** "5도면 코트나 패딩이 필요하다."
6. **최종 답변:** "현재 서울은 5도이므로 코트나 가벼운 패딩을 추천합니다."

**Agent와 Tools의 상호작용**

1. **Agent가 입력을 받음**: 사용자의 요청을 LLM으로 분석.
2. **적합한 Tool 선택**: 요청을 처리하는 데 가장 적합한 Tool을 선택.
3. **Tool 실행 및 결과 반환**: Tool을 실행하고 결과를 받아 사용자에게 응답.

In [1]:
%pip install langchain_openai langchain-community langchain-tavily langgraph wikipedia numexpr arxiv ddgs -Uqqq

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv  # .env 파일의 환경변수 로드
import os                       # 환경변수 접근용

load_dotenv()                                                         # 현재 위치의 .env를 읽어와 환경변수로 등록
os.environ["OPENAI_API_KEY"] = os.getenv("openai_key")                # .env의 openai_key 값을 OPENAI_API_KEY로 등록
os.environ["LANGSMITH_TRACING"] = 'true'                              # LangSmith 트레이싱 활성화
os.environ["LANGSMITH_ENDPOINT"] = 'https://api.smith.langchain.com'  # LangSmith API 엔드포인트 설정
os.environ["LANGSMITH_PROJECT"] = 'skn23-langchain'                   # LangSmith 프로젝트명 설정
os.environ["LANGSMITH_API_KEY"] = os.getenv("langsmith_key")          # .env의 langsmith_key 값을 LANGSMITH_API_KEY로 등록
os.environ["TAVILY_API_KEY"] = os.getenv("tavily_key")                 


In [ ]:
import importlib, pkgutil  # 모듈 동적 로드 / 패키지 탐색 유틸

package = importlib.import_module('langchain_community.tools')  # 패키지 로드

for module in pkgutil.iter_modules(package.__path__):           # 패키지 경로 아래의 하위 모듈들을 순회
    print(module.name)                                          # 각 모듈의 이름 출력

ainetwork
amadeus
arxiv
asknews
audio
azure_ai_services
azure_cognitive_services
bearly
bing_search
brave_search
cassandra_database
clickup
cogniswitch
connery
convert_to_openai
databricks
dataforseo_api_search
dataherald
ddg_search
e2b_data_analysis
edenai
eleven_labs
few_shot
file_management
financial_datasets
github
gitlab
gmail
golden_query
google_books
google_cloud
google_finance
google_jobs
google_lens
google_places
google_scholar
google_search
google_serper
google_trends
graphql
human
ifttt
interaction
jina_search
jira
json
memorize
merriam_webster
metaphor_search
mojeek_search
multion
nasa
nuclia
office365
openai_dalle_image_generation
openapi
openweathermap
passio_nutrition_ai
playwright
plugin
polygon
powerbi
pubmed
render
requests
riza
scenexplain
searchapi
searx_search
semanticscholar
shell
slack
sleep
spark_sql
sql_database
stackexchange
steam
steamship_image_generation
tavily_search
vectorstore
wikidata
wikipedia
wolfram_alpha
yahoo_finance_news
you
youtube
zapier
zenguar

In [ ]:
from langchain_community.tools import WikipediaQueryRun         # 위키피디아 질의 실행 TOOL
from langchain_community.utilities import WikipediaAPIWrapper   # 위키피디아 API 래퍼(검색/요약)

Wiki_tool = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper()) # 위키피디아 API 래퍼를 TOOL에 연결
Wiki_tool.run("피지컬 AI")                                       # 위키 검색/요약 결과 출력

'Page: G-Dragon\nSummary: Kwon Ji-yong (Korean: 권지용; born August 18, 1988), best known as G-Dragon (지드래곤), is a South Korean rapper, singer, songwriter, and entrepreneur. Dubbed the "King of K-pop", he rose to prominence as the leader of the South Korean boy band BigBang, which went on to become one of the best-selling boy bands in the world.\nBorn in Seoul, he made his public debut at age six as part of the musical group Little Roo\'ra. In 2009, he released his first solo studio album Heartbreaker; the album and its title track of the same name were commercially successful, becoming the best-selling album by a Korean soloist at the time and earning him Album of the Year at the 2009 Mnet Asian Music Awards. This was followed by a collaboration with bandmate T.O.P for the album GD & TOP in 2010. His first EP, One of a Kind (2012), earned Best Album at the 2013 Seoul Music Awards and spawned the chart-topping single "That XX". In 2013, he embarked on the One of a Kind World Tour, his fir

In [ ]:
from langchain.chat_models import init_chat_model
from langchain.agents import create_agent
from pprint import pprint

messages = [
    ('human', '올데이 프로젝트 멤버 알려줘!')    
]

llm = init_chat_model("openai:gpt-4.1-mini")
#print(llm.invoke(messages))


agent = create_agent(
    model = llm,
    tools = [Wiki_tool]    
)

response = agent.invoke({'messages' : messages})

pprint(response)


content='올데이 프로젝트(All Day Project)는 대한민국의 힙합 크루로, 여러 아티스트들이 참여하는 프로젝트입니다. 올데이 프로젝트의 멤버로는 주로 다음과 같은 아티스트들이 있습니다:\n\n- 딥플로우 (Deepflow)\n- 더 콰이엇 (The Quiett)\n- 개코 (Gaeko)\n- 코드 쿤스트 (Code Kunst)\n- 펀치넬로 (Punchnello)\n- 우탄 (Woo)\n- 넉살 (Nucksal)\n\n멤버 구성이나 참여 아티스트는 활동 시기에 따라 달라질 수 있으니, 최신 정보를 원하시면 공식 채널이나 최근 발표된 내용을 참고하는 것이 좋습니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 153, 'prompt_tokens': 17, 'total_tokens': 170, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_28996bb172', 'id': 'chatcmpl-D5osN5WUswHSumysk7RMEbtlkRXA2', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019c2ce2-b061-7c80-95ca-95d5648a1ffd-0' tool_calls=[] invalid_tool_calls=[] usage_metadat

In [13]:
response['messages'][3].content

'올데이 프로젝트(AllDay Project, ADP)는 대한민국의 혼성 그룹으로, The Black Label이 기획하고 관리하는 팀입니다. 멤버는 Annie, Tarzzan, Bailey, Woochan, 그리고 Youngseo 총 5명으로 구성되어 있습니다.'

### load_tools

**load_tools 사용가능 목록**

라이브러리를 통해 제공되는 외부 tool들을 langchain-community에서 통합하여 사용할 수 있다.
웬만한 기능들은 langchain 생태계 내에서 쉽게 쓸 수 있다.

https://docs.langchain.com/oss/python/integrations/providers/overview

https://docs.langchain.com/oss/python/integrations/tools

| 도구 이름        | 기능 예시               |
|------------------|------------------------|
| llm-math         | LLM 기반 수학 계산     |
| wikipedia        | 위키백과 검색          |
| serpapi          | 구글 검색 API          |
| requests_get     | HTTP GET 요청          |
| requests_post    | HTTP POST 요청         |
| arxiv            | arXiv 논문 검색        |
| pubmed           | PubMed 논문 검색       |
| dalle            | DALL-E 이미지 생성     |
| bing_search      | Bing 검색              |
| duckduckgo_search| DuckDuckGo 검색        |

### arxiv

In [18]:
from langchain_community.agent_toolkits.load_tools import load_tools

llm = init_chat_model("openai:gpt-4.1-mini")
tools = load_tools(['llm-math', 'wikipedia'], llm=llm)
agent = create_agent(
    llm,
    tools,
    system_prompt="""
당신은 현명한 챗봇입니다.
주어진 도구를 적절히 활용하여, 사용자의 질문에 거짓없는 답변을 작성해주세요.
단, 숫자계산은 반드시 llm-math 도구를 사용해야 합니다.
"""        
)

response = agent.invoke({'messages' : "3.5의 3제곱은 몇이야? 그리고 그 결과에 5를 곱해줘"})
pprint(response)
print(response['messages'][3].content)

{'messages': [HumanMessage(content='3.5의 3제곱은 몇이야? 그리고 그 결과에 5를 곱해줘', additional_kwargs={}, response_metadata={}, id='dd0b6205-39e2-4122-b42f-4e2428193f7f'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 193, 'total_tokens': 213, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_82dbabdb2c', 'id': 'chatcmpl-D5pCMlQce6myT3EAyQ8QZmlWM6XbE', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c2cf5-9819-7062-b5e4-3f005af922a0-0', tool_calls=[{'name': 'Calculator', 'args': {'__arg1': '3.5^3'}, 'id': 'call_XWmjXyKPuobYwGAv8ledw3JA', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_to

### duckduckgo
https://docs.langchain.com/oss/python/integrations/tools/ddg

DuckDuckGo는 개인정보 추적 없이 웹 검색을 제공하는 검색 엔진으로,
LangChain에서는 이를 외부 최신 정보 검색용 Tool로 활용한다.

실시간 웹 검색 가능
→ LLM의 knowledge cutoff 이후 이슈(뉴스, 화제, 트렌드)에 대응 가능
로그인/API 키 불필요
→ 실습·교육 환경에서 바로 사용 가능
프라이버시 중심
→ 사용자 검색 이력 추적 없음

LangChain에서 제공하는 DuckDuckGo Tool 차이
- DuckDuckGoSearchRun
    - 검색 결과를 하나의 텍스트 요약으로 반환
    - 빠른 질의응답용에 적합
- DuckDuckGoSearchResults
    - 검색 결과를 리스트(제목, 링크, 스니펫 등 구조화) 형태로 반환
    - 에이전트가 여러 결과를 비교·판단해야 할 때 유리

In [ ]:
# DDG 검색 Tool 2종류
from langchain_community.tools import DuckDuckGoSearchRun, DuckDuckGoSearchResults

ddgs = DuckDuckGoSearchRun()        # 검색결과를 텍스트 요약 형태로 반환
print(ddgs.invoke("현재 한국에서 가장 핫한 인물은?"))


ddgs2 = DuckDuckGoSearchResults()    # 검색 결과를 구조화 된 리스트로 반환
print(ddgs2.invoke("현재 한국에서 가장 핫한 인물은?"))

Oct 28, 2025 · 2025년 기준, 한국에서 가장 주목받는 배우·가수·예능인 30인의 고향, 학력, 데뷔 계기, 대표 에피소드까지 한눈에 정리했습니다.2025년 현재 가장 핫 한 한국 연예인 30명의 프로필을 한눈에! 15 마동석 영화배우 마동석은1971년 한국에서 태어나 18살 (1989)에 가족을 따라 미국으로 이민을 갔으며 미국 시민권을 취득했다. 미국에서 퍼스널 트레이너로 활동하다 2002년 영화 [천군] 오디션에 붙은 뒤 본격적으로 한국에서 배우 활동을 시작한다. 지금 가장 핫 한 스타는 누구? 2025년 최신 기준, 개인 계정 보유자 중 팔로워 수가 가장 많은 한국 연예인 TOP 10을 소개합니다! 공식 그룹 계정 (BTS, BLACKPINK)은 제외하고, 개인 활동 중심으로 선정했어요. 지금부터 10위부터 1위까지 역순으로 확인해볼까요? 👇 2025년 가장 핫 한 연예인은 누구? 여배우 TOP3부터 장기 인기 스타 유재석·방탄소년단, 글로벌 활약 이수혁, 전역 예정 몬스타엑스·뷔까지 총정리. 지금 확인하세요! Dec 30, 2024 · 전체 선수 중 수비 실책 30개로 가장 많았다. 해외 5, 트럼프+머스크+푸틴+오타니+스위프트 4년 만에 다시 대통령 권좌로 돌아온 도널드 트럼프 당선인.
snippet: August 12, 2025 - '주요 소득' 부문은 음원, 공연, 영화·방송 출연료, 광고, 연봉으로 한정하며 '언론노출' 부문은 국내 5대 종합지의 헤드라인에 언급된 빈도수를 인물별로 뉴스빅데이터 분석서비스 '빅카인즈'에서 추출한다. '소셜미디어' 부문은 셀럽들이 가장 많이 이용하는 인스타그램과 유튜브의 팔로워 수를 4월 1일 기준으로 조사해 합산한다., title: 포브스 코리아 파워 셀러브리티 40 - 위키백과, 우리 모두의 백과사전, link: https://ko.wikipedia.org/wiki/포브스_코리아_파워_셀러브리티_40, snippet: August 14, 2025 - 《한국을 빛낸 100명의 위인들》(韓國을 빛낸 百